# Tools
* A tool in CrewAI is a skill or function that agents can utilize to perform various actions.
* This includes tools from the CrewAI Tools and LangChain Tools.

## How to Use CrewAI Tools
* Begin by installing our extra tools package. Write in your terminal `pip install 'crewai[tools]'`
* All CrewAI tools are built with error handling capabilities and support caching.
* Here’s an example demonstrating their use:

In [ ]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key
os.environ["OPENAI_API_KEY"] = "Your Key"

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst's summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
crew.kickoff()

## How to use LangChain Tools
You can integrate [LangChain tools](https://python.langchain.com/docs/integrations/tools/) into your CrewAI applications to enhance your agents' capabilities. Here's a step-by-step guide on how to do this:

#### Step 1: Install Required Packages
Ensure you have both `crewai` and `langchain` installed:

```bash
pip install crewai langchain
```

#### Step 2: Set Up Environment Variables
Some LangChain tools require API keys. For example, if you're using the `GoogleSerperAPIWrapper`, set the `SERPER_API_KEY` in your environment:

```bash
export SERPER_API_KEY=your_serper_api_key
```

#### Step 3: Create a LangChain Tool Wrapper
Wrap the LangChain tool using CrewAI's `BaseTool` class:

```python
from crewai.tools import BaseTool
from pydantic import Field
from langchain_community.utilities import GoogleSerperAPIWrapper

class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries."
    search: GoogleSerperAPIWrapper = Field(default_factory=GoogleSerperAPIWrapper)

    def _run(self, query: str) -> str:
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"
```


This wrapper allows CrewAI agents to utilize the LangChain tool seamlessly.

#### Step 4: Assign the Tool to a CrewAI Agent
Create a CrewAI agent and assign the wrapped tool:

```python
from crewai import Agent

researcher = Agent(
    role='Research Analyst',
    goal='Gather current market data and trends',
    backstory="""You are an expert research analyst with years of experience in
    gathering market intelligence. You're known for your ability to find
    relevant and up-to-date market information and present it in a clear,
    actionable format.""",
    tools=[SearchTool()],
    verbose=True
)
```

#### Step 5: Create a Task and Execute
Define a task for the agent and execute it:

```python
from crewai import Task, Crew

task = Task(
    description='Find and summarize the latest AI news',
    expected_output='A bullet list summary of the top 5 most important AI news',
    agent=researcher
)

crew = Crew(
    agents=[researcher],
    tasks=[task],
    verbose=True
)

result = crew.kickoff()
print(result)
```

This setup enables your CrewAI agent to perform tasks using the integrated LangChain tool. 

#### Tips and Best Practices

- **Error Handling**: Implement robust error handling within your tool wrappers to manage API failures or unexpected responses.

- **Caching**: Use caching mechanisms to store frequent queries and reduce API calls.

- **Tool Arguments**: Leverage the flexibility of tool arguments to customize the functionality of each tool.

By following these steps, you can effectively integrate LangChain tools into your CrewAI applications, enhancing the capabilities of your AI agents.

## Creating your own tools
There are two main ways for one to create a CrewAI tool:
* Subclassing.
* Using the @tool decorator.

#### Subclassing

In [ ]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    argument: str = Field(..., description="Description of the argument.")

class MyCustomTool(BaseTool):
    name: str = "Name of my tool"
    description: str = "What this tool does. It's vital for effective utilization."
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, argument: str) -> str:
        # Your tool's logic here
        return "Tool's result"

#### Using the @tool decorator

In [ ]:
from crewai.tools import tool
@tool("Name of my tool")
def my_tool(question: str) -> str:
    """Clear description for what this tool is useful for, your agent will need this information to use it."""
    # Function logic here
    return "Result from your custom tool"

## Custom Caching Mechanism
Tools can optionally implement a cache_function to fine-tune caching behavior. This function determines when to cache results based on specific conditions, offering granular control over caching logic.

In [ ]:
from crewai.tools import tool

@tool
def multiplication_tool(first_number: int, second_number: int) -> str:
    """Useful for when you need to multiply two numbers together."""
    return first_number * second_number

def cache_func(args, result):
    # In this case, we only cache the result if it's a multiple of 2
    cache = result % 2 == 0
    return cache

multiplication_tool.cache_function = cache_func

writer1 = Agent(
        role="Writer",
        goal="You write lessons of math for kids.",
        backstory="You're an expert in writing and you love to teach kids but you know nothing of math.",
        tools=[multiplication_tool],
        allow_delegation=False,
    )
    #...

## Using other 3rd Party Tools
* In the CrewAI Documentation page you can find a list of other 3rd Party Tools you can use with your CrewAI apps. It will be interesting for your to take a quick look at them.
* You will see that many of them are related with RAG tasks.